<a href="https://colab.research.google.com/github/cagBRT/promptEngineering/blob/main/gpt2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -U -q transformers

In [ ]:
from transformers import pipeline, set_seed
generator = pipeline('text-generation', model='gpt2')
set_seed(42)
generator("Hello, I'm a language model,", max_length=30, num_return_sequences=5)


In [ ]:
from transformers import pipeline, set_seed
generator = pipeline('text-generation', model='gpt2')
set_seed(42)
generator("The man worked as a", max_length=10, num_return_sequences=5)

In [ ]:

set_seed(42)
generator("The dog worked as a", max_length=10, num_return_sequences=5)


https://huggingface.co/gpt2